In [1]:
import newspaper
import pandas as pd
from tqdm import tqdm

In [ ]:
data = pd.read_csv("data/2023_04_05_matriz_not_julia.csv", sep=";")
data

In [3]:
tqdm.pandas()

# Función para extraer el contenido de la noticia
def get_article_text(url):
    try:
        article = newspaper.Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"Error al procesar {url}: {e}")
        return None  # Devuelve None si hay un error

In [ ]:
# Crear la nueva columna "contenido"
data['contenido'] = data['enlace'].progress_apply(get_article_text)

# Ver los primeros resultados
print(data.head())

In [5]:
data.to_csv("data/contenido_2023_04_05_matriz_not_julia.csv")

In [ ]:
# Corregir URLs con "htmll" al final
data['enlace'] = data['enlace'].str.replace("htmll", "html", regex=False)

# Filtrar las filas donde 'contenido' es None o está vacío
filt = data['contenido'].isna() | (data['contenido'] == "")

# Aplicar la función solo a estas filas
data.loc[filt, 'contenido'] = data.loc[filt, 'enlace'].progress_apply(get_article_text)

In [7]:
data.to_csv("data/contenido_2023_04_05_matriz_not_julia.csv")

In [ ]:
data

In [ ]:
import re
import pandas as pd
import spacy
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

# Descargar stopwords de NLTK si no lo has hecho
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))  # Stopwords en español

# Cargar modelo de Spacy para lematización en español
nlp = spacy.load("es_core_news_sm")

def clean_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'\s+', ' ', text)  # Eliminar espacios extra
    text = re.sub(r'http\S+', '', text)  # Eliminar URLs
    text = re.sub(r'\W', ' ', text)  # Eliminar caracteres no alfanuméricos
    
    # Tokenización y lematización con Spacy
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct]
    
    return " ".join(tokens).strip()

# Aplicar limpieza
data['contenido_clean'] = data['contenido'].apply(clean_text)


In [ ]:
data.to_csv("data/contenido_2023_04_05_matriz_not_julia.csv")